# KPMG TEST NOTEBOOK

Selenium is a tool initially created to automate tests on websites. It is therefore very useful when information is accessible by clicking on links. A button for example is an element from which it is very difficult to obtain the link. BeautifulSoup then becomes limited.
In this case, use Selenium.

In [2]:

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from random import randrange
import pandas as pd
import html2text

In [3]:

df = pd.read_pickle("../data/Staatsblad_nl_fr.pkl") 
print(df['cleantextnl'][0])

  
BRUSSELS HOOFDSTEDELIJK GEWEST  
---  
  
### _8 JANUARI 2020. - Ministerieel besluit tot vaststelling van de modellen
van formulier bedoeld in artikel 8, § 1 van het besluit van de Brusselse
Hoofdstedelijke Regering van 19 januari 2017 tot vaststelling van de
modaliteiten van het gunstregime toepasselijk op de successierechten bij
overdracht van familiale ondernemingen en familiale vennootschappen _

  
  
De Minister van de Brusselse Hoofdstedelijke Regering, belast met Financiën en
Begroting,  
Gelet op het Wetboek der Successierechten, artikel 60bis/2, § 1, ingevoegd
door de ordonnantie van 12 december 2016 houdende het tweede deel van de
fiscale hervorming;  
Gelet op het besluit van de Brusselse Hoofdstedelijke Regering van 19 januari
2017 tot vaststelling van de modaliteiten van het gunstregime toepasselijk op
de successierechten bij overdracht van familiale ondernemingen en familiale
vennootschappen, artikel 8;  
Gelet op de gelijkekansentest uitgevoerd in toepassing van art

In [48]:
#!python -m spacy download en_core_news_md

import pandas as pd
import spacy
import re

# Load the model
nlp = spacy.load("nl_core_news_md")
stopwords = nlp.Defaults.stop_words

def dict_sort(dict_uniqwords:dict ) -> dict:
    from operator import itemgetter
    srt=sorted(dict_uniqwords.items(), key=itemgetter(1),reverse=True)
    return srt

def getsomestats(dict_uniqwords:dict ,txt_original:str, topcnt:int = 15):
    #---NOT USED FOR NOW

    #print("@@@@@@@@@@@@@@@@@@@@",dict_uniqwords)
    # total number of unique words in doc
    print("unique word count :", len(dict_uniqwords))

    # total number of words in doc
    wordcnt=0
    for s in dict_uniqwords: wordcnt+=dict_uniqwords[s]
    print("total wordcount: ",wordcnt)
    
    #sorted words count
    ###from operator import itemgetter
    ###srt=sorted(dict_uniqwords.items(), key=itemgetter(1),reverse=True)
    srt = dict_sort(dict_uniqwords)

    res=[]
    for i in range(topcnt):
        try:
            res.append(srt[i][0])
            print(srt[i])
        except:
            break
    #print original doc
    #print(txt_original)
    return res

def nlp_cleanandlemmatize(txtdoc: str, list_wordstoskip:str = ''):
    LANG='nl'
    # Load the model
    #nlp = spacy.load("nl_core_news_md")
    #stopwords = nlp.Defaults.stop_words

    # words to discard
    months={'nl':['januari','februari','maart','april','mei','juni','augustus','september','oktober','november','december'],
            'fr':['janvier','fevrier','mars','avril','mai','juin','juillet','aout','septembre','octobre','decembre']}
    days={'nl':['maandag','dinsdag','woensdag','donderdag','vrijdag','zaterdag','zondag'],
            'fr':['lundi','mardi','mercredi','jeudi','vendredi','samedi','dimanche']}
    # 
    nlp.max_length=10000000
    nlp_doc=nlp(txtdoc)
    list_allwordslemmatized=[]
    dict_uniqwords={}
    list_tokens=[]
    #sequential filter
    for token in nlp_doc:
        lemma_lower=token.lemma_.lower()
        if token in stopwords:
            continue
        if (token.is_punct or token.is_space or token.is_stop):
            continue
        if token.text.isdecimal():
            continue
        if True in [char.isdigit() for char in token.text]:
            continue
        if token.text[-1] == '.':
            continue
        if len(token.text) <= 2:
            continue
        if lemma_lower in months[LANG]:
            continue
        if lemma_lower in days[LANG]:
            continue
        if lemma_lower in list_wordstoskip:
            continue
        #pass only nouns
        #print(token.pos_,token.pos)
        if token.pos_ != 'NOUN':
            continue
        #create dict of unique words with count
        if lemma_lower not in dict_uniqwords: 
            dict_uniqwords[lemma_lower]=1
            #save tokens for vector comparison
            list_tokens.append(token)
        else:
            dict_uniqwords[lemma_lower]+=1
        ##save tokens for vector comparison
        ##list_tokens.append(token)

        
        #saving the words (lemma equates number   -   lemma_ equates word)
        ###list_allwordslemmatized.append([token.lemma,lemma_lower,token.text])
    return dict_uniqwords, list_tokens
    #end of filter-----------------------------------------------

def concat_docs(docs:list[str] ,nmbr:int = 0) -> str:
    #concat nmbr dcocuments from doc return txtdoc
    #nmbr=10 #test anz docus
    txtdoc=''
    #for txt in df['cleantextnl'].values:
    for txt in docs:
        txtdoc+=str(txt)
        if nmbr == 1:
            break
        nmbr-=1
    return txtdoc


df = pd.read_pickle("../data/Staatsblad_nl_fr.pkl") 

print("Top 15:\n#######")
numberofdocumenttoscan=10
for n in range(numberofdocumenttoscan):
    #how many docs to use for testing
    #txtdoc = concat_docs(df['cleantextnl'].values, 10)
    txtdoc = df['cleantextnl'].values[n]

    list_wordstoskip=['artikel','document'] #add here words to discard

    dict_uniqwords,list_tokens=nlp_cleanandlemmatize(txtdoc,list_wordstoskip)

    list_sorted_uniqwords = dict_sort(dict_uniqwords)

    ###print("Remaining words after filtering: ",len(list_allwordslemmatized))
    #topcnt=10 
    #list_word_topcnt=getsomestats(dict_uniqwords,txtdoc,topcnt)
    #print("Top",topcnt,": ",list_word_topcnt)
    ##################################################################
    print("---------------------------------------------------")

    ##################################################################
    # Output:   list_sorted_uniqwords , dict_uniqwords, list_tokens
    #
    print(list_sorted_uniqwords[0:10])


Top 15:
#######
---------------------------------------------------
[('besluit', 8), ('successierecht', 6), ('regering', 5), ('gunstregime', 5), ('overdracht', 5), ('formulier', 4), ('onderneming', 4), ('vennootschap', 4), ('wetboek', 4), ('vaststelling', 3)]
---------------------------------------------------
[('besluit', 4), ('tarief', 4), ('vertaling', 4), ('januar', 4), ('minister', 4), ('dienst', 3), ('sätzen', 3), ('wörter', 3), ('wijziging', 2), ('vaststelling', 2)]
---------------------------------------------------
[('gutes', 17), ('nicht', 9), ('auf', 9), ('ke/estgb', 7), ('kostenlose', 6), ('zurverfügungstellung', 6), ('vorteil', 6), ('minister', 5), ('goed', 4), ('vertaling', 4)]
---------------------------------------------------
[('onderwijs', 14), ('promotie', 13), ('afdeling', 9), ('wetenschap', 7), ('specialisatie', 6), ('besluit', 6), ('code', 5), ('type', 4), ('gebied', 3), ('decreet', 3)]
---------------------------------------------------
[('besluit', 9), ('ministe

In [50]:
#compare a token against a tokenslist
def token_compare(token_totest,list_tokens,min_score:int = 60):
    list_tokens_toreturn=[]
    for token in list_tokens:
        #print("token_compare",token)
        similar=token_totest.similarity(token)
        #print(token_totest, "<->", token, similar)
        if similar >= min_score/100:
            list_tokens_toreturn.append([token_totest,token,similar])
    return list_tokens_toreturn

numberofdocumenttoscan=5
for n in range(numberofdocumenttoscan):
    txtdoc = df['cleantextnl'].values[n]
    list_wordstoskip=['artikel','document'] #add here words to discard
    dict_uniqwords,list_tokens=nlp_cleanandlemmatize(txtdoc,list_wordstoskip) #get tokens
    #list_sorted_uniqwords = dict_sort(dict_uniqwords) #sort
    
    print(n,"--------------------------")
    test=nlp('belasting')
    res=token_compare(test,list_tokens,65)
    for i in res:
        print(i,'\n')
    


0 --------------------------
[belasting, modaliteiten, 0.6629545770614875] 

[belasting, successierechten, 0.9043948372239012] 

[belasting, modaliteiten, 0.6629545770614875] 

1 --------------------------
[belasting, belasting, 1.0] 



C:\Users\bmadmin\AppData\Local\Temp\ipykernel_10112\2716612306.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similar=token_totest.similarity(token)


2 --------------------------
3 --------------------------
4 --------------------------
[belasting, inkomstenbelastingen, 0.8263346384300361] 

[belasting, interesten, 0.7032102702518628] 



In [ ]:
print(df_all2['cleantextnl'][728])


In [ ]:
import html2text


#df['cleantextnl']=''
#df['cleantextfr']=''
pnt=0
for language in ['fr','nl']:
    
    for url in df['Link NL']:
        htmllanguage='language='+language
        url=url.replace('language=nl',htmllanguage)
        url2='view-source:'+url
        url2=url2.replace('.pl?','_body.pl?')
        driver = webdriver.Firefox()
        driver.get(url2)
        #simulate human 
        rnd=randrange(1,2)
        sleep(rnd)
        page=driver.page_source
        driver.close()

        soup = BeautifulSoup(page, "html").text

        dummy=html2text.html2text(soup)
        dummy=dummy.split('---|---|---|---|---|---')
        
        dummy=dummy[1].split('begin |  |  eerste woord |  laatste woord |  |')
        cleantext=dummy[0]
        #print(cleantext)
        #save 
        txtlabel='cleantext'+language
        df[txtlabel][pnt]=cleantext
        pnt+=1
        #if pnt == 1:
        #    pnt=0
        #    break
        

df.to_pickle("../data/Staatsblad.pkl")  
unpickled_df = pd.read_pickle("../data/Staatsblad.pkl") 
unpickled_df.head()


In [ ]:
soup

In [ ]:
#!pip install html2text
import html2text

a=html2text.html2text(soup)
a=a.split('---|---|---|---|---|---')
a=a[1].split('begin |  |  eerste woord |  laatste woord |  |')
print(a[0])

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

b=a

b=b.split('\n')

 
for i in b:
   
    print(remove_html_tags(i),'\n')
 
 



### Load libraries

If you are missing any libraries in the next cell, you'll need to install them before continuing.

In [ ]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os

date = strftime("%Y-%m-%d")

### Install Selenium according to this manual

https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium/bin

*NB: On Linux, put your `geckodriver` (the downloaded extension) in the equivalent path on your machine into `/home/<YOUR_NAME>/.local/bin/`*

We will simulate a search on the official Python website.

In [ ]:
import selenium

# The selenium.webdriver module provides all the implementations of WebDriver
# Currently supported are Firefox, Chrome, IE and Remote. The `Keys` class provides keys on
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Here, we create instance of Firefox WebDriver.
#driver = webdriver.Firefox()
driver = webdriver.Firefox()

# The driver.get method will lead to a page given by the URL. WebDriver will wait until the page is fully
# loaded (i.e. the "onload" event has been triggered) before returning the control to your script.
# It should be noted that if your page uses a lot of AJAX calls when loading, WebDriver may not know
# when it was fully loaded.
driver.get("http://www.python.org")

# The following line is a statement confirming that the title contains the word "Python".
assert "Python" in driver.title

# WebDriver offers several methods to search for items using one of the methods
# `find_element_by_...` .
# For example, the input text element can be located by its name attribute by
# using the `find_element_by_name` method.
elem = driver.find_element_by_name("q")

# Then we send keys. This is similar to entering keys using your keyboard.
# Special keys can be sent using the `Keys` class imported in line 7 (from selenium.webdriver.common.keys import Keys).
# For security reasons, we will delete any pre-filled text in the input field
# (for example, "Search") so that it does not affect our search results:
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)

# After submitting the page, you should get the result if there is one. To ensure that certain results
# are found, make an assertion:
assert "No results found." not in driver.page_source
try:
    assert "Trademark Usage Policy" in driver.page_source
except:
    print("ok I found this text \"Trademark Usage Policy\"")


driver.close()

#### Open the source code of the webpage and check that the search area (field) is called "q".

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
try: 
    elem = driver.find_element_by_name("q")
    print("element q has been found!")
except:
    print("element q has NOT been found!")
    driver.close()

driver.close()

### Getting a phone number from *leboncoin*

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

#url = "https://www.leboncoin.fr/sports_hobbies/1536839557.htm/"
url = "https://www.leboncoin.fr/sports_hobbies/2112572252.htm"
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(url)

python_button = driver.find_elements_by_xpath('//div[@data-reactid="269"]')[0]
python_button.click()
input("Waiting...")
# And then we use Beautiful soup
soup = BeautifulSoup(driver.page_source)

driver.close()

for elem in soup.find_all("a", attrs={"data-qa-id": "adview_number_phone_contact"}):
    print(elem.text)

### Starting from *leboncoin*, collect all the information available to define the product being sold. Use `selenium` for the telephone number.

### API (Application Program Interface)

A set of tools and methods that allow different applications to interact with each other. In the case of a web service, we can retrieve data dynamically. By using an API correctly, we can thus obtain in real time, the modifications made on a "parent" site.

For example, we will retrieve online news, for example from the "L'équipe" website.

Follow the instructions at https://newsapi.org/s/lequipe-api to retrieve an "API key" connection key

Your API key is: `73bbb95f8ecb49b499113a46481b4af1`


It is frequent that a key does not work after a while (e.g. `5 min`n `30 min`, a day, ...)
So don't jump up if you get an error message back.

In [ ]:
import requests

key = "73bbb95f8ecb49b499113a46481b4af1"
url = "https://newsapi.org/v2/top-headlines?sources=lequipe&apiKey=" + key
response = requests.get(url)

# Here the response format is a json file, it is used as a dictionary
print(response.json())

In [ ]:
dictionnary = response.json()
print(dictionnary.keys())

In [ ]:
for element in list(dictionnary.keys()):
    print("##############################################")
    print("Key: ", element, "// Values: ", dictionnary[element])

In [ ]:
# And now we have lists in dictionaries(it's a JSON file actually but it's very similar)
# We will discover the information of the article key.

for element in enumerate(dictionnary["articles"]):
    print("###############################################")
    print(element)

In [ ]:
# So if we keep going, it gives us another dictionary!
for element in dictionnary["articles"][0].keys():
    print(" Key : ", element, "Values : ", dictionnary["articles"][0][element])

### Make a script that allows you to take details of the last ten news from the team or another site. Store them in a nice CSV or excel file.

In [ ]:
import pandas as pd
newslist=[]
for element in dictionnary["articles"]:
    #print(element.keys())
    
    newslist.append([element['publishedAt'],element['content']])
    newslist.sort(reverse=True)

#for i in newslist:
#    print(i[0]," ***************** ",i[1])
    
print(type(newslist),len(newslist))
publishedat=[]
content=[]

for i in newslist:
    publishedat.append(i[0])
    content.append(i[1])
#print(len(publishedat),len(content))
#print(publishedat)
#print(content)
#break
df = pd.DataFrame({"publishedAt": publishedat})
df["Content"] = content   
#print(df)

df.to_csv("./assets/orderedNewsList.csv", index=False)
